## Notebook perparation

### Load (pre-import) jupyter extensions

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

In [ ]:
import warnings
import math
import re
from tempfile import TemporaryDirectory
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import dateutil
from datetime import timezone
from zoneinfo import ZoneInfo
import bokeh
from astropy.utils.iers import IERSDegradedAccuracyWarning

In [ ]:
from astropy.time import Time, TimeDelta, TimezoneInfo
import astropy.coordinates

In [ ]:
from rubin_sim.scheduler.example import example_scheduler
from rubin_sim.scheduler import sim_runner
from rubin_sim.scheduler.model_observatory import ModelObservatory

In [ ]:
from uranography.api import HorizonMap, ArmillarySphere

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
%aimport schedview
%aimport schedview.app.prenight
%aimport schedview.compute.scheduler
%aimport schedview.compute.survey
%aimport schedview.collect.scheduler_pickle
%aimport schedview.plot.survey

### Load (post-import) jupyter extensions

Load the `panel` extension so we can see plots in this notebook.

In [ ]:
pn.extension()

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="astropy.coordinates",
    message=r"Tried to get polar motions for times after IERS data is valid..*",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings("ignore", category=IERSDegradedAccuracyWarning, append=True)

## Create sample `scheduler` and `conditions` instances

We can either construct instances of `scheduler` and `conditions` instances "from scratch," or load them from an existing python `pickle`.

In normal operations, the observatory infrastructure will construct the instances, and observatory staff loads them from archived pickles.

In this notebook, though, we may need to construct such a pickle for ourselves.

Pick a name for the file we will be using.
If you remake the scheduler, the current contents of the file will be overwritten!

In [ ]:
scheduler_pickle = "example_scheduler.p.xz"

# If the scheduler pickle name isn't set, invent a temporary one
# that will get cleaned up automatically.
if "scheduler_pickle" not in locals():
    temp_dir = TemporaryDirectory()
    temp_dir_path = Path(temp_dir.name)
    scheduler_pickle = str(temp_dir_path.joinpath("example_scheduler.p.xz"))

If you need to create the pickle, set `remake_scheduler_pickle` to `True` the first time you run this notebook, which will make the pickle.
In general, you'll probably prefer to load this pickle, which is faster, so set it back to `False` after you run it once.

In [ ]:
remake_scheduler_pickle = not Path(scheduler_pickle).exists()

if remake_scheduler_pickle:
    local_timezone = ZoneInfo("Chile/Continental")
    start_time = Time(pd.Timestamp("2025-08-01 22:00:00", tzinfo=local_timezone))
    current_time = Time(pd.Timestamp("2025-08-01 23:00:00", tzinfo=local_timezone))

    results = schedview.compute.scheduler.create_example(
        current_time,
        start_time,
        scheduler_pickle_fname=scheduler_pickle,
    )
    del results

Now, load the pickle:

In [ ]:
scheduler, conditions = schedview.collect.scheduler_pickle.read_scheduler(
    scheduler_pickle
)

## Calling the scheduler

In [ ]:
type(conditions)

The `conditions` object, an instance of `rubin_sim.scheduler.features.conditions.Conditions`, provides an interface to the environmental data required to choose an observation.
These data include the date and time (expressed as a [modified Julian date, or MJD](https://en.wikipedia.org/wiki/Julian_day)).
This date can be set or viewed in a conventional format using the `astropy.time.Time` class:

In [ ]:
Time(conditions.mjd, format="mjd").iso

In [ ]:
conditions.mjd = Time("2025-08-02 03:01:00").mjd
Time(conditions.mjd, format="mjd").iso

In [ ]:
type(scheduler)

The state of the `scheduler` object (an instance of `rubin_sim.scheduler.schedulers.core_scheduler.CoreScheduler`) can the be set according to these conditions, at which point it can select observations for these conditions:

In [ ]:
scheduler.update_conditions(conditions)
observations = scheduler.request_observation()
observations

The `observation` object returned by `scheduler.request_observation()` is a `numpy.recarray` with a list of (one) observation, the observation for the time requestied.
It contains the data needed to actually take the observation.

Jupyter does not display `numpy.recarray` nicely, but can be converted to a `pandas.DataFrame` and the relevant columns extracted:

In [ ]:
pd.DataFrame(observations)[
    [
        "RA",
        "dec",
        "rotSkyPos_desired",
        "filter",
        "exptime",
        "nexp",
        "flush_by_mjd",
        "note",
    ]
]

## Understanding the scheduler: how the scheduler selects an observation

When `request_observation` is called, `scheduler` checks if there are any valid observations in its `queue` member, and if there are, it returns the first one.

If there are not, it tries to fill the `queue` using objects in its `survey_lists` member, which is a list of lists of instances of subclasses of `BaseSurvey`:

In [ ]:
scheduler.survey_lists

Instances of `BaseSurvey` and its subclasses all have two fundamental methods:
- `calc_reward_function(conditions: Conditions) -> float | numpy.array`, which returns a "reward" for observations from this survey under these conditions. If a value of `-inf` is returned, that indicates that the survey is not feasible under the provided conditions.
- `generate_observations(conditions: Conditions) -> numpy.recarray`, which returns observations that would be selected by this survey for these conditions.

Each element of `survey_lists` is a tier, a list of surveys with equal priority.

The `scheduler` iterates over the list of tiers in `survey_lists` until it finds one for which there is at least one survey that returns a valid reward (one greater than `-np.inf`):

```
rewards = None
for ns, surveys in enumerate(self.survey_lists):
    rewards = np.zeros(len(surveys))
    for i, survey in enumerate(surveys):
        rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))
    # If we have a good reward, break out of the loop
    if np.nanmax(rewards) > -np.inf:
        self.survey_index[0] = ns
        break
```

If it finds one, it uses the survey in that list that provides the maximum reward to fill the queue:

```
if (np.nanmax(rewards) == -np.inf) | (np.isnan(np.nanmax(rewards))):
    self.flush_queue()
else:
    to_fix = np.where(np.isnan(rewards) == True)
    rewards[to_fix] = -np.inf
    # Take a min here, so the surveys will be executed in the order they are
    # entered if there is a tie.
    self.survey_index[1] = np.min(np.where(rewards == np.nanmax(rewards)))
    # Survey return list of observations
    result = self.survey_lists[self.survey_index[0]][
        self.survey_index[1]
    ].generate_observations(self.conditions)

    self.queue = result        rewards = None
```

If the `queue` is still empty after the attempt to fill it, then `generate_observations` returns `None`.

# Examine the scheduler

Additional detail on how `scheduler` will fill its queue is available with `scheduler.make_reward_df`:

In [ ]:
reward_df = scheduler.make_reward_df(conditions)

The return from `scheduler.make_reward_df` can be summarized with `schedview.compute.scheduler.make_scheduler_summary_df`:

In [ ]:
schedview.compute.scheduler.make_scheduler_summary_df(scheduler, conditions, reward_df)

## Examining surveys

### The summary table for a survey

Each survey in the list of lists in `scheduler.survey_lists` is an instance of a subclass of `rubin_sim.scheduler.surveys.base_survey.BaseSurvey`.
Some subclasses of `BaseSurvey` can provide additional data on their calculation of rewards and select of observations.
In particular, `BlobSurvey` and `GreedySurvey` are subclasses of `BaseMarkovSurvey`, which computes rewards through a weighted sum of a set of basis functions, and examination of these basis functions provides valuable information for understanding why they behave the way they do.

To further explore a specific survey, first extract the instance of interest from `scheduler.survey_lists`:

In [ ]:
tier_id, survey_id = 2, 3
survey = scheduler.survey_lists[tier_id][survey_id]

Then, use `schedview.compute.survey.make_survey_reward_df` to extract the needed data from `reward_df` (computed above):

In [ ]:
schedview.compute.survey.make_survey_reward_df(
    survey, conditions, reward_df.loc[[(tier_id, survey_id)], :]
)

Each row listed summarizes one basis function.

The reward returned by any give basis function can either be an array (a healpix map) or a scalar.

If the reward is an array (a healpix map of the sky), the `max_basis_reward` and `basis_area` show the maximum reward over the sky and the area of the sky with a reward > `-np.inf`.

The `max_accum_reward` and the `accum_area` show the maximum reward and feasible area of all basis functions at that point in the list and above.

### Maps of basis functions

Finally, we can show the full maps for basis functions that provide them.

Set `nside` to the `healpix` `nside` at which you want the maps displayed. This can be lower than the native `nside` for the maps, in which case the maps will be shown at reduced resolution.

Then, use `schedview.compute.survey.compute_maps` to collect the maps for all basis functions into a dictionary:

In [ ]:
nside = 16
survey_maps = schedview.compute.survey.compute_maps(survey, conditions, nside=nside)
survey_maps.keys()

Now, you can select the key for the map you want to look at, and use `schedview.plot.survey.map_survey_healpix` to display the map:

In [ ]:
healpix_key = "g_sky"
sky_map = schedview.plot.survey.map_survey_healpix(
    conditions.mjd, survey_maps, healpix_key, nside
)
sky_map.notebook_display()

## Putting it all together

In [ ]:
def show_scheduler(
    pickle_fname, mjd, tier_id=2, survey_id=2, map_key="reward", nside=None
):
    scheduler, conditions = schedview.collect.scheduler_pickle.read_scheduler(
        pickle_fname
    )

    if nside is None:
        nside = conditions.nside

    # Set the date
    conditions.mjd = mjd
    scheduler.update_conditions(conditions)

    # Get data on survey and basis funtions
    reward_df = scheduler.make_reward_df(conditions)
    survey = scheduler.survey_lists[tier_id][survey_id]
    survey_maps = schedview.compute.survey.compute_maps(survey, conditions, nside=nside)

    # Display the selected map
    sky_map = schedview.plot.survey.map_survey_healpix(
        conditions.mjd, survey_maps, healpix_key, nside
    )
    sky_map.notebook_display()

    # Display the selected survey basis function summary
    survey_reward_df = schedview.compute.survey.make_survey_reward_df(
        survey, conditions, reward_df.loc[[(tier_id, survey_id)], :]
    )
    display(survey_reward_df)

    # Summarize rewards from all surveys
    scheduler_summary_df = schedview.compute.scheduler.make_scheduler_summary_df(
        scheduler, conditions, reward_df
    )
    display(scheduler_summary_df)


mjd = Time(
    pd.Timestamp("2025-08-01 23:00:00", tzinfo=ZoneInfo("Chile/Continental"))
).mjd
show_scheduler(scheduler_pickle, mjd, 2, 2, "reward")

## Desired functionality in a dashboard

Most of this is done already in the `sched_maps` dashboard, but this needs to be refactored.

Needed functionality:

 - Clear layout, headings, and key.
 - A text entry box for a file path or URL for a pickle to load.
 - A date entry box to select the date in a user-friendly format (then to be converted to MJD)
 - Drop-downs (or similar) to select the tier, survey, and map to plot.
 - Clicking on the row for a basis function in the survey_reward_df table should cause that map to be shown in the map figure.
 - Clicking on the row for survey in the in the scheduler_summary_df table should cause that survey to be shown both the table and map figure.
 - URLs in the survey_reward_df and scheduler_summary_df tables should not be shown as urls, but rather be linked to by the basis_function and survey_name text.
 - Any errors need to be reported to the user.
 - Operations that take a long time (loading a new pickle, changing the date) should disable the interface and show a status message during the update.

Needed soon:

- Accept parameters (pickle file url, mjd, survey, nside) in the url so other dashboards can link to it in a given state.

Even better:

- A section box to display maps with different nsides. (nsides can be 8, 16, or 32)
- Tools for changing the color scheme for the map